In [1]:
#Importing Libraries
# import and commit
import os
import io
import numpy
import pandas as pd
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [2]:
#Reading Source files
def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message

In [3]:
#Appending the source files into one Dataframe
def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = pd.concat([data, dataFrameFromDirectory("emails/spam", "spam")]);
data = pd.concat([data, dataFrameFromDirectory("emails/ham", "ham")])

In [4]:
data.head()

,message,class
emails/spam/00249.5f45607c1bffe89f60ba1ec9f878039a,"Dear Homeowner,\n\n \n\nInterest Rates are at ...",spam
emails/spam/00373.ebe8670ac56b04125c25100a36ab0510,ATTENTION: This is a MUST for ALL Computer Use...,spam
emails/spam/00214.1367039e50dc6b7adb0f2aa8aba83216,This is a multi-part message in MIME format.\n...,spam
emails/spam/00210.050ffd105bd4e006771ee63cabc59978,IMPORTANT INFORMATION:\n\n\n\nThe new domain n...,spam
emails/spam/00033.9babb58d9298daa2963d4f514193d7d6,This is the bottom line. If you can GIVE AWAY...,spam


In [5]:
data.describe()

,message,class
count,3000,3000
unique,2943,2
top,URL: http://www.askbjoernhansen.com/archives/2...,ham
freq,4,2500


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3000 entries, emails/spam/00249.5f45607c1bffe89f60ba1ec9f878039a to emails/ham/00135.bd3bc1c036eab89c9c50cff40958c939
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   message  3000 non-null   object
 1   class    3000 non-null   object
dtypes: object(2)
memory usage: 70.3+ KB


In [7]:
#Finding the percentage of spam and non-spam emails in the source files
no_spam=(data['class'].value_counts()['spam'])
no_ham=(data['class'].value_counts()['ham'])
print('No. of spam emails =',no_spam)
print('No. of non-spam emails =',no_ham)
spam_percent= (no_spam/len(data))*100
ham_percent= (no_ham/len(data))*100
print('Spam email Percentage =',round(spam_percent,2),'%')
print('Non-spam email Percentage =',round(ham_percent,2),'%')

No. of spam emails = 500
No. of non-spam emails = 2500
Spam email Percentage = 16.67 %
Non-spam email Percentage = 83.33 %


In [8]:
target_values = data['class'].values

In [9]:
# CountVectorizer to split up messages into wordlist
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)

In [10]:
classifier = MultinomialNB()
classifier.fit(counts, target_values)

MultinomialNB()

In [11]:
#Predicting with the classifier
examples = ['Free Free Free!']
example_counts = vectorizer.transform(examples)
output = classifier.predict(example_counts)
output

array(['spam'], dtype='<U4')